# Adaptative + Behavioural fine-tuning 🤖⚙️

Okay, at this point we have already trained the base-bert-cased on our own dataset following the adaptative framework (MLM objective). Now, our model is supposed to be more adjusted to our specific medical domain in terms of semantics.

In this experiment, we will further adjust our model to the objective task. This is, a round of behavioural fine-tuning over an already adapted backbone! Lets see what happens.

<figure style='text-align:center';>
  <img src="../data/images/A+BFT.png">
  
  <figcaption>
  Adaptative + Behavioural fine-tuning schema 
  </figcaption>
</figure>

Important points:
* Dataset: [medical_questions_pairs](https://huggingface.co/datasets/medical_questions_pairs)
* Model: [bert-base-cased](https://huggingface.co/bert-base-cased)
* We will define auxiliar functions in auxiliar.py file
* We will be logging the results in Weight&Biases.
<br>

In [1]:
import torch
import config

if torch.cuda.is_available():
   device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [2]:
device

device(type='cuda', index=0)

## 1. Data preparation

### 1.1. Import and set creation

Import data and create partitions.

In [2]:
from datasets import load_dataset

# Download and extract data
data = load_dataset("medical_questions_pairs")
data = data['train']

# Split it
data = data.train_test_split(test_size=0.07, seed=config.SEED)

  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label'],
        num_rows: 2834
    })
    test: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label'],
        num_rows: 214
    })
})

As we can see, there is not that much ammount of samples. We will have to take that into consideration when training the models.

### 1.2. Tokenize and encode data

As mentioned, we will use **bert-base-cased** tokenizer.

**NOTE: If we had created a new version of the tokenizer during our adaptative fine-tuning (by adding new tokens to the vocab), we would need to use our new models checkpoint to load this tokenizer. We didn't so we can still use bert-base-cased.**

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(config.checkpoint, use_fast=True)

In [4]:
data = data.map(lambda x: tokenizer(x['question_1'], x['question_2'], truncation=True, padding='max_length'), batched=True)

In [5]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 2. Adapted backbone + Behavioural fine-tuning 

Okay! Time to train the model.

Now, we will download our previously trained backbone from Weight&Biases and load it with an untrained classification head. 

For this experiment we are not freezing the encoder's parameters. We will train everything in order to adapt the whole thing to the objetive task.

In [6]:
import wandb
from tempfile import TemporaryDirectory
from transformers import AutoModelForSequenceClassification

# Download artifact to a temp dir
with TemporaryDirectory() as temp_dir:
  run = wandb.init()
  artifact = run.use_artifact('jjceamoran/fine-tuning-mlms/encoder:v0', type='model')
  artifact_dir = artifact.download(temp_dir)

  model = AutoModelForSequenceClassification.from_pretrained(artifact_dir, num_labels=2)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jjceamoran. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact encoder:v0, 1240.91MB. 11 files... 
wandb:   11 of 11 files downloaded.  
Done. 0:0:4.5
Some weights of the model checkpoint at /tmp/tmp4k00m412 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
So

### 3.1. Init WandB

In [7]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [8]:
run_name = 'adaptative_and_behavioural_training'
notes = "This experiment consists on a full head + backbone training on the objective task. We will use a domain-adapted backbone."
run = wandb.init(project='fine-tuning-mlms',
           name=run_name,
           notes=notes,
           job_type='train')


In [9]:
from transformers import Trainer, TrainingArguments
from training_aux import compute_metrics
import sklearn

training_args = TrainingArguments(
    output_dir="./experiments/" + run_name,
    learning_rate=3e-5, # low learning rate.
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to='wandb',
    run_name=run_name
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [10]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: question_2, question_1, dr_id. If question_2, question_1, dr_id are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2834
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1775
  Number of trainable parameters = 108311810
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "t

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.476078,0.799065
2,0.450400,0.686071,0.789720
3,0.195000,0.886721,0.803738
4,0.195000,0.871546,0.836449
5,0.068800,0.898074,0.836449


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: question_2, question_1, dr_id. If question_2, question_1, dr_id are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 214
  Batch size = 8
Saving model checkpoint to ./experiments/adaptative_and_behavioural_training/checkpoint-355
Configuration saved in ./experiments/adaptative_and_behavioural_training/checkpoint-355/config.json
Model weights saved in ./experiments/adaptative_and_behavioural_training/checkpoint-355/pytorch_model.bin
tokenizer config file saved in ./experiments/adaptative_and_behavioural_training/checkpoint-355/tokenizer_config.json
Special tokens file saved in ./experiments/adaptative_and_behavioural_training/checkpoint-355/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertF

TrainOutput(global_step=1775, training_loss=0.20431049642428545, metrics={'train_runtime': 1305.0063, 'train_samples_per_second': 10.858, 'train_steps_per_second': 1.36, 'total_flos': 3728283654451200.0, 'train_loss': 0.20431049642428545, 'epoch': 5.0})

In [11]:
# Log model

artifact = wandb.Artifact('classifier', type='model')
artifact.add_dir('./experiments/adaptative_and_behavioural_training/checkpoint-1775')
wandb.log_artifact(artifact)

wandb: Adding directory to artifact (./experiments/adaptative_and_behavioural_training/checkpoint-1775)... Done. 6.7s
